# Simulating Fluid Flow using Neural Networks

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

## Implementing 2D Models

### Basic CNN

In [ ]:
def basic_cnn(input_shape, out_c, n_filters=8):
    input = keras.Input(input_shape)
      
    x = layers.Conv2D(n_filters, (3, 3), padding='same', activation='relu')(input)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling2D(padding='same')(x)
    
    x = layers.Conv2D(n_filters * 2, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters * 2, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling2D(padding='same')(x)
    
    x = layers.Conv2D(n_filters * 4, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters * 4, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2DTranspose(n_filters * 2, 2, 2, padding='same')(x)
    
    x = layers.Conv2D(n_filters * 2, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(n_filters * 2, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2DTranspose(n_filters, 2, 2, padding='same')(x)
    
    output = layers.Conv2D(out_c, (1, 1), padding='same')(x)
    
    name = 'basic_cnn{n_filters}'.format(n_filters=n_filters)
    
    return keras.Model(input, output, name=name)

In [ ]:
def basic_cnn(input_shape, out_c, n_filters=8):
    input = keras.Input(input_shape)
      
    x = layers.Conv2D(n_filters, 3, padding='same', activation='relu')(input)
    x = layers.Conv2D(n_filters, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling2D(padding='same')(x)
    
    x = layers.Conv2D(n_filters * 2, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(n_filters * 2, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling2D(padding='same')(x)
    
    x = layers.Conv2D(n_filters * 4, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(n_filters * 4, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2DTranspose(n_filters * 2, 2, 2, padding='same')(x)
    
    x = layers.Conv2D(n_filters * 2, 3, padding='same', activation='relu')(x)    
    x = layers.Conv2D(n_filters * 2, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2DTranspose(n_filters, 2, 2, padding='same')(x)
    
    x = layers.Conv2D(n_filters * 2, 3, padding='same', activation='relu')(x)    
    x = layers.Conv2D(n_filters * 2, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    output = layers.Conv2D(out_c, 1, padding='same')(x)
    
    name = 'basic_3d_cnn{n_filters}'.format(n_filters=n_filters)
    
    return keras.Model(input, output, name=name)

In [ ]:
basic_cnn((64, 64, 4), 3, 8).summary()

### UNet

In [ ]:
def unet_conv_block(x, n_filters):
    x = layers.Conv2D(
        n_filters, (3, 3), padding='same', activation='relu'
    )(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(
        n_filters, (3, 3), padding='same', activation='relu'
    )(x)
    x = layers.BatchNormalization()(x)
    
    return x

In [ ]:
def unet_down(x, n_filters):
    x = unet_conv_block(x, n_filters)
    skip = layers.MaxPooling2D(padding='same')(x)
    return x, skip

In [ ]:
def unet_up(x, skip, n_filters):
    x = layers.Conv2DTranspose(
        n_filters, 2, 2, padding='same'
    )(x)
    x = layers.Concatenate()([x, skip])
    x = unet_conv_block(x, n_filters)
    return x

In [ ]:
def unet(input_shape, out_c, n_filters=8):
    input = keras.Input(input_shape)
    
    # downsampling
    d1, p1 = unet_down(input, n_filters)
    d2, p2 = unet_down(p1, n_filters*2)
    d3, p3 = unet_down(p2, n_filters*4)
    d4, p4 = unet_down(p3, n_filters*8)
    
    # bottleneck
    b = unet_conv_block(p4, n_filters*16)
    
    # upsampling
    u1 = unet_up(b, d4, n_filters*8)
    u2 = unet_up(u1, d3, n_filters*4)
    u3 = unet_up(u2, d2, n_filters*2)
    u4 = unet_up(u3, d1, n_filters)
    
    output = layers.Conv2D(out_c, (1, 1), padding='same')(u4)
    
    name = 'unet{n_filters}'.format(n_filters=n_filters)
    
    return keras.Model(input, output, name=name)

In [ ]:
unet((64, 64, 4), 3, 8).summary()

## Implementing 3D Models

### Basic 3D CNN

In [ ]:
def basic_3d_cnn(input_shape, out_c, n_filters=8):
    input = keras.Input(input_shape)
      
    x = layers.Conv3D(n_filters, 3, padding='same', activation='relu')(input)
    x = layers.Conv3D(n_filters, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling3D((1, 2, 2), padding='same')(x)
    
    x = layers.Conv3D(n_filters * 2, 3, padding='same', activation='relu')(x)
    x = layers.Conv3D(n_filters * 2, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.MaxPooling3D((1, 2, 2), padding='same')(x)
    
    x = layers.Conv3D(n_filters * 4, 3, padding='same', activation='relu')(x)
    x = layers.Conv3D(n_filters * 4, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3DTranspose(n_filters * 2, 2, 2, padding='same')(x)
    
    x = layers.Conv3D(n_filters * 2, 3, padding='same', activation='relu')(x)    
    x = layers.Conv3D(n_filters * 2, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3DTranspose(n_filters, 2, 2, padding='same')(x)
    
    x = layers.Conv3D(n_filters, 3, padding='same', activation='relu')(x)    
    x = layers.Conv3D(n_filters, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    output = layers.Conv3D(out_c, 1, (8, 1, 1), padding='same')(x)
    
    name = 'basic_3d_cnn{n_filters}'.format(n_filters=n_filters)
    
    return keras.Model(input, output, name=name)

In [ ]:
basic_3d_cnn((2, 64, 64, 4), 1, 8).summary()

### CNN LSTM

In [ ]:
def cnn_lstm(input_shape, out_c, n_filters=8):
    input = layers.Input(input_shape)

    x = layers.ConvLSTM2D(
        filters=n_filters,
        kernel_size=3,
        padding="same",
        return_sequences=True,
        activation="relu",
    )(input)
    x = layers.BatchNormalization()(x)
    
    x = layers.ConvLSTM2D(
        filters=n_filters*2,
        kernel_size=3,
        padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.ConvLSTM2D(
        filters=n_filters*4,
        kernel_size=3,
        padding="same",
        return_sequences=True,
        activation="relu",
    )(x)
    
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(
        filters=n_filters*2, kernel_size=3, activation="relu", padding="same"
    )(x)
    x = layers.Conv3D(
        filters=n_filters, kernel_size=3, activation="relu", padding="same"
    )(x)
    
    x = layers.BatchNormalization()(x)
    
    output = layers.Conv3D(
        filters=out_c, kernel_size=1, strides=(2, 1, 1), padding="same"
    )(x)

    # Next, we will build the complete model and compile it.
    name = 'cnn_lstm{n_filters}'.format(n_filters=n_filters)
    return keras.Model(input, output, name=name)

In [ ]:
cnn_lstm((2, 64, 64, 4), 1, 8).summary()

## Loading Data

In [ ]:
fpath = 'data/sim_np/size64/sim_512x64x64x64x3.npy'
dataset = np.load(fpath)
fpath = 'data/sim_np/size64/bound_64x64.npy'
boundary = np.load(fpath)

# # Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(dataset.shape[0])
np.random.shuffle(indexes)
train_index = indexes[: int(0.9 * dataset.shape[0])]
val_index = indexes[int(0.9 * dataset.shape[0]) :]
train_dataset = dataset[train_index]
val_dataset = dataset[val_index]

print("Training Dataset Shape:", train_dataset.shape)
print("Validation Dataset Shape:", val_dataset.shape)

### Creating 2D Training Data

In [ ]:
def unroll_2d(x):
    return x.reshape(x.shape[0]*x.shape[1], x.shape[2], x.shape[3], x.shape[4])

def shift_frames_2d(data, boundary):
    x = np.zeros((data.shape[0], data.shape[1] - 1, data.shape[2], data.shape[3], data.shape[4] + 1), np.float16)
    y = np.zeros((data.shape[0], data.shape[1] - 1, data.shape[2], data.shape[3], data.shape[4]), np.float16)
    boundary = np.expand_dims(boundary, axis=-1)
    
    for i in range(data.shape[0]):
        for j in range(data.shape[1] - 1):
            
            x[i, j] = np.concatenate((data[i, j], boundary), axis=-1)
            y[i, j] = data[i, j + 1]
        
    return unroll_2d(x), unroll_2d(y)

In [ ]:
# apply the processing function to the datasets.
x_train_2d, y_train_2d = shift_frames_2d(train_dataset, boundary)
x_val_2d, y_val_2d = shift_frames_2d(val_dataset, boundary)

print("2D Training Dataset Shapes: " + str(x_train_2d.shape) + ", " + str(y_train_2d.shape))
print("2D Validation Dataset Shapes: " + str(x_val_2d.shape) + ", " + str(y_val_2d.shape))

### Create 3D Training Data

In [ ]:
def shift_frames_3d(data, boundary):
    N = data.shape[1] - N_PREV_FRAMES
    x = np.zeros((data.shape[0]*N, N_PREV_FRAMES, data.shape[2], data.shape[3], 4), np.float16)
    y = np.zeros((data.shape[0]*N, 1, data.shape[2], data.shape[3], 3), np.float16)
    
    boundary = np.expand_dims(boundary, -1)
    
    for i in range(data.shape[0]):
        for j in range(N):
            # append all the frames to a single list
            frames = []
            for f in range(N_PREV_FRAMES):
                frames.append(np.concatenate((data[i, j+f, :, :, :], boundary), -1))
            
            x[i*N+j] = np.stack(frames, 0)
            y[i*N+j] = np.expand_dims(data[0, j+N_PREV_FRAMES, :, :, :], (0))

    return x, y

In [ ]:
N_PREV_FRAMES = 2
x_train_3d, y_train_3d = shift_frames_3d(train_dataset, boundary)
x_val_3d, y_val_3d = shift_frames_3d(val_dataset, boundary)

print("3D Training Dataset Shapes: " + str(x_train_3d.shape) + ", " + str(y_train_3d.shape))
print("3D Validation Dataset Shapes: " + str(x_val_3d.shape) + ", " + str(y_val_3d.shape))

## Training

In [ ]:
# define modifiable training hyperparameters.
epochs = 10
batch_size = 128

# define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=0)

# create model to train
# model = unet((64, 64, 4), 3, 16)
model = cnn_lstm(x_train_3d[0].shape, 3, 8)
model.compile(
    loss=keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(),
)

# fit data to model
history = model.fit(
    x_train_3d,
    y_train_3d,
    batch_size=batch_size,
    epochs=epochs,
    validation_data = (x_val_3d, y_val_3d),
    callbacks=[early_stopping, reduce_lr],
)

In [ ]:
model.save('models/keras/{name}'.format(name=model.name))
# model = keras.models.load_model('models/keras/unet16')

## Animate the Labels and Predictions

In [ ]:
import matplotlib.pyplot as plt  
import matplotlib.animation as animation
import os 
%matplotlib inline

FPS = 24
INTERVAL = 1000.0/FPS
# NUM_EXAMPLES = val_dataset.shape[0]
NUM_EXAMPLES = 16
MODEL_NAME = model.name

### Animating 2D Models

In [ ]:
def format_2d(x, boundary):
    return np.concatenate((x, np.expand_dims(boundary, (0, -1))), -1)

In [ ]:
for ex_i in range(NUM_EXAMPLES):
    x = val_dataset[ex_i]
    
    # CREATE THE LABEL VIDEO
    fig = plt.figure()
    label_ims = []
    for i in range(x.shape[0] - 1):
        plt.axis('off')
        label_im = plt.imshow(np.rot90(x[i+1, :, :, 0]))
        label_ims.append([label_im])
    
    dir = 'videos/{name}/ex_{ex}/'.format(name=MODEL_NAME, ex=ex_i)
    os.makedirs(dir, exist_ok=True)
    ani = animation.ArtistAnimation(fig, label_ims, interval=INTERVAL, blit=True, repeat_delay=1000)
    ani.save(dir+'water_label_64.mp4')

    # CREATE THE MODEL'S VIDEO
    pred_ims = []
    x_i = format_2d(np.expand_dims(x[0], 0), boundary)
    for i in range(x.shape[0] - 1):
        y = model.predict(x_i)
        plt.axis('off')
        pred_im = plt.imshow(np.rot90(y[0, :, :, 0]))
        pred_ims.append([pred_im])
        
        x_i = format_2d(y, boundary)

    ani = animation.ArtistAnimation(fig, pred_ims, interval=INTERVAL, blit=True, repeat_delay=1000)
    ani.save(dir+'water_pred_64.mp4')
    
    plt.close()

### Animating 3D Model

In [ ]:
def format_3d(x, y, i, boundary):
    # append all the frames to a single list
    frames = []
    for f in range(N_PREV_FRAMES-1):
        frames.append(np.concatenate((x[i+f, :, :, :], boundary), -1))
    frames.append(np.concatenate((y, boundary), -1))
    return np.stack(frames, 0)


In [ ]:
for ex_i in range(NUM_EXAMPLES):
    x = val_dataset[ex_i]
    
    # CREATE THE LABEL VIDEO
    fig = plt.figure()
    label_ims = []
    for i in range(x.shape[0]-N_PREV_FRAMES):
        plt.axis('off')
        label_im = plt.imshow(np.rot90(y[i+N_PREV_FRAMES, 0, :, :, 0]))
        label_ims.append([label_im])
    
    dir = 'videos/{name}/ex_{ex}/'.format(name=MODEL_NAME, ex=ex_i)
    os.makedirs(dir, exist_ok=True)
    ani = animation.ArtistAnimation(fig, label_ims, interval=INTERVAL, blit=True, repeat_delay=1000)
    ani.save(dir+'water_label_64.mp4')

    # CREATE THE MODEL'S VIDEO
    pred_ims = []
    x_i = np.expand_dims(x[0], 0)
    for i in range(x.shape[0]-N_PREV_FRAMES):
        pred = model.predict(x_i)
        plt.axis('off')
        pred_im = plt.imshow(np.rot90(pred[i, 0, :, :, 0]))
        pred_ims.append([pred_im])
        
        x_i = np.concatenate((y, np.expand_dims(boundary, (0, -1))), -1)

    ani = animation.ArtistAnimation(fig, pred_ims, interval=INTERVAL, blit=True, repeat_delay=1000)
    ani.save(dir+'water_pred_64.mp4')
    
    plt.close()